In [2]:
from utils import load_series_dfs
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import os
from shapely import wkt, Point
import geopandas as gpd
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import pickle
import requests

In [12]:
gpkg_path = os.path.join('..', 'data', 'df_mapping_centroids.gpkg')
df_mapping = gpd.read_file(gpkg_path)

df_mapping.columns

Index(['nemesys_key', 'geom_g', 'centroid_lon', 'centroid_lat', 'geometry'], dtype='object')

In [4]:
# 1. Resource-ID für stündliche Klimastationen
resource_id = 'klima-v2-1h'

# 2. Metadata-Endpoint ansprechen
url_meta  = f"https://dataset.api.hub.geosphere.at/v1/station/historical/{resource_id}/metadata"
resp_meta = requests.get(url_meta)
resp_meta.raise_for_status()

# 3. Liste der Station-Dicts rausziehen
data = resp_meta.json()          # {'stations': [ {...}, {...}, … ]}
stations_list = data['stations']

# 4. In DataFrame umwandeln
stations_df = pd.DataFrame.from_records(stations_list)

# 5. Wichtige Spalten anzeigen
print("Spalten:", stations_df.columns.tolist())
print(stations_df[['id','name','lat','lon']].head())

Spalten: ['type', 'id', 'group_id', 'name', 'state', 'lat', 'lon', 'altitude', 'valid_from', 'valid_to', 'has_sunshine', 'has_global_radiation', 'is_active']
   id              name       lat       lon
0   1            Aflenz  47.54594  15.24069
1   2  Aigen im Ennstal  47.53278  14.13826
2   3       Allentsteig  48.69083  15.36694
3   4         Amstetten  48.10889  14.89500
4   5        Bad Aussee  47.61050  13.75844


In [14]:
# 1. stations_df → GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.lon, stations_df.lat),
    crs="EPSG:4326"
)

# 2. df_mapping CRS prüfen/setzen
# (falls df_mapping.geometry schon Points in EPSG:4326 enthält, sonst entsprechend anpassen)
df_mapping = df_mapping.set_crs("EPSG:4326", allow_override=True)

# 3. In metrisches CRS projizieren für Meter-Abstände
df_map_m = df_mapping.to_crs(epsg=3857)
stations_m  = stations_gdf.to_crs(epsg=3857)

# 4. Nearest Join mit Entfernungs­spalte 'dist_m'
joined = gpd.sjoin_nearest(
    df_map_m,
    stations_m,
    how="left",
    distance_col="dist_m",
    max_distance=50000  # optional: nur Stationen ≤ 50 km
)

joined

,nemesys_key,geom_g,centroid_lon,centroid_lat,geometry,index_right,type,id,group_id,name,state,lat,lon,altitude,valid_from,valid_to,has_sunshine,has_global_radiation,is_active,dist_m
0,FL_00025241,MULTIPOLYGON (((576611.379498681 5233885.47556...,571866.668405,5.237265e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FL_00352191,MULTIPOLYGON (((585315.470109405 5225725.82005...,589874.782901,5.237858e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FL_00020896,MULTIPOLYGON (((442366.697798016 5277755.65335...,449815.252870,5.278740e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FL_00024730,MULTIPOLYGON (((458466.07960983 5343618.499366...,459947.407067,5.339844e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FL_00024735,MULTIPOLYGON (((455813.69099947 5345132.902266...,456514.181913,5.342561e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FL_00352244,MULTIPOLYGON (((445856.374990906 5365743.91906...,439780.694796,5.351455e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FL_00352430,MULTIPOLYGON (((484559.218368485 5193207.33573...,488959.046206,5.183190e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,FL_00024731,MULTIPOLYGON (((449827.987549417 5340014.16225...,455734.427577,5.326785e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FL_00352054,MULTIPOLYGON (((168517.508101939 5271056.46775...,179254.145161,5.263559e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FL_00352189,MULTIPOLYGON (((604720.060484524 5244360.40220...,625275.533165,5.289817e+06,POINT (Infinity Infinity),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 1. Prüfe, welches CRS deine Centroid-Points aktuell haben:
print(df_mapping.crs)

# 2. Schau dir die X- und Y-Werte mal numerisch an:
print(df_mapping.geometry.x.describe())
print(df_mapping.geometry.y.describe())

# 3. Wenn df_mapping.crs nicht 'EPSG:4326' ist,
#    setze es (falls deine lon/lat-Spalten wirklich in Grad sind):
df_mapping = df_mapping.set_crs("EPSG:4326", allow_override=True)

EPSG:4326
count        43.000000
mean     475667.354531
std      116524.997176
min      103863.919233
25%      413445.175856
50%      459558.991970
75%      578439.737755
max      652374.756802
dtype: float64
count    4.300000e+01
mean     5.287221e+06
std      6.784214e+04
min      5.157201e+06
25%      5.239969e+06
50%      5.305620e+06
75%      5.337090e+06
max      5.396672e+06
dtype: float64


In [31]:
import numpy as np
from sklearn.neighbors import BallTree

# 1. Arrays in Radianten umwandeln
coords_map  = np.deg2rad(df_mapping[['centroid_lat','centroid_lon']])
coords_stat = np.deg2rad(stations_df[['lat','lon']])

# 2. Baum bauen und je 1 Nachbarn suchen
tree = BallTree(coords_stat, metric='haversine')
dist_rad, idx = tree.query(coords_map, k=1)

# 3. Abstand in km
dist_km = (dist_rad.flatten() * 6371.0)

# 4. Ergebnisse ins df_mapping schreiben
df_mapping['nearest_stat_id'] = stations_df.iloc[idx.flatten()]['id'].values
df_mapping['dist_km']         = dist_km

print(df_mapping[[
    'nemesys_key','nearest_stat_id','dist_km'
]].head())

   nemesys_key  nearest_stat_id       dist_km
0  FL_00025241             1000  16106.058151
1  FL_00352191              206   9364.231281
2  FL_00020896             1000   7784.572573
3  FL_00024730            11146  16690.586989
4  FL_00024735              501  12201.077685


In [20]:
df_mapping[[
    'nemesys_key','nearest_stat_id','dist_km'
]]

,nemesys_key,nearest_stat_id,dist_km
0,FL_00025241,1000,16106.058151
1,FL_00352191,206,9364.231281
2,FL_00020896,1000,7784.572573
3,FL_00024730,11146,16690.586989
4,FL_00024735,501,12201.077685
5,FL_00352244,501,8187.715259
6,FL_00352430,206,15152.008496
7,FL_00024731,218,16783.331677
8,FL_00352054,14120,6369.591851
9,FL_00352189,14120,9452.158689


In [21]:
# 4. Stations-GeoDataFrame anlegen (CRS: EPSG:4326)
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.lon, stations_df.lat),
    crs="EPSG:4326"
)

# 5. Beide ins Meter-CRS bringen
df_m  = df_mapping.to_crs(epsg=3857)
stat_m = stations_gdf.to_crs(epsg=3857)

# 6. Nearest-Join – Abstand in Metern in der Spalte 'dist_m'
nearest = gpd.sjoin_nearest(
    df_m, stat_m,
    how="left",
    distance_col="dist_m",
    max_distance=50_000   # z.B. nur bis 50 km
)

# Ergebnis: pro Centroid bekommst du die nächstgelegene Station
# plus den exakten Meter-Abstand in 'dist_m'.
print(nearest[[
    'nemesys_key','id','name','dist_m'
]].head())

   nemesys_key  id name  dist_m
0  FL_00025241 NaN  NaN     NaN
1  FL_00352191 NaN  NaN     NaN
2  FL_00020896 NaN  NaN     NaN
3  FL_00024730 NaN  NaN     NaN
4  FL_00024735 NaN  NaN     NaN


In [22]:
print("Aktuelles CRS:", df_mapping.crs)
print("Beispiele Geometrie:", df_mapping.geometry.head())
print("X-Werte (sollten etwa 13…17 sein):",
      df_mapping.geometry.x.min(), df_mapping.geometry.x.max())
print("Y-Werte (sollten etwa 47…49 sein):",
      df_mapping.geometry.y.min(), df_mapping.geometry.y.max())

Aktuelles CRS: EPSG:4326
Beispiele Geometrie: 0      POINT (574298.47349 5236470.316)
1    POINT (592254.42841 5234861.03802)
2    POINT (451102.75767 5278928.05818)
3    POINT (459558.99197 5339463.04333)
4    POINT (456008.64107 5342557.63837)
Name: geometry, dtype: geometry
X-Werte (sollten etwa 13…17 sein): 103863.91923287 652374.7568024495
Y-Werte (sollten etwa 47…49 sein): 5157201.42221358 5396672.28395662


In [24]:
df_mapping = df_mapping.set_crs("EPSG:31287", allow_override=True)
print("Neues CRS:", df_mapping.crs)

Neues CRS: EPSG:31287


In [25]:
# nach WGS84 (Lon/Lat) – so siehst Du zur Kontrolle nun vernünftige 13…17 / 47…49-Werte
df_wgs84 = df_mapping.to_crs(epsg=4326)
print(df_wgs84.geometry.x.min(), df_wgs84.geometry.x.max())
print(df_wgs84.geometry.y.min(), df_wgs84.geometry.y.max())

# direkt in Meter-CRS (EPSG:3857), um Entfernungen in Metern zu berechnen
df_m = df_mapping.to_crs(epsg=3857)

-9.633129594324927 33.961030867891296
85.41732403161878 86.59842170864616


In [28]:
stations_m = stations_gdf.to_crs(epsg=3857)

nearest = gpd.sjoin_nearest(
    df_m, stations_m,
    how="left",
    distance_col="dist_m",

)

print(nearest[["nemesys_key","id","name","dist_m"]].head())

   nemesys_key   id      name        dist_m
0  FL_00025241  501  Litschau  1.483983e+07
1  FL_00352191  501  Litschau  1.481343e+07
2  FL_00020896  501  Litschau  1.526900e+07
3  FL_00024730  501  Litschau  1.581969e+07
4  FL_00024735  501  Litschau  1.585031e+07


In [30]:
nearest.id.unique()

array([501])